# Calculate monthly mean data from daily inputs

## imports

In [4]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import os
import xcdat as xc

# set data paths

In [5]:
obsPath = "/p/user_pub/PCMDIobs/obs4MIPs_input/RSS/RSS-MW5-1/v20230605/"

# lazy load data

In [6]:
dataPath = os.path.join(obsPath, "*.nc")
print("dataPath:", dataPath)
ds = xc.open_mfdataset(dataPath)

dataPath: /p/user_pub/PCMDIobs/obs4MIPs_input/RSS/RSS-MW5-1/v20230605/*.nc


## view dataset

In [ ]:
ds